In [12]:
import numpy as np
import inspect
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.circuit.library import QFT

def create_function_unitary(f, n):
    """Create a unitary matrix representing the function f"""
    Uf = np.zeros((2**n, 2**n))
    for x in range(2**n):
        y = f(x)
        Uf[x, y] = 1
    return Uf

def period_finding_circuit(f):
    """Create a quantum circuit to detect the period of a function f"""
    # Determine the number of input bits required for f
    n = len(inspect.signature(f).parameters)

    # Create quantum registers
    qr_input = QuantumRegister(n)
    qr_output = QuantumRegister(n)
    cr = ClassicalRegister(n)

    # Create the circuit
    qc = QuantumCircuit(qr_input, qr_output, cr)

    # Apply Hadamard gates to the input register
    qc.h(qr_input)

    # Apply the function f as a unitary operator
    Uf = create_function_unitary(f, n)
    qc.unitary(Uf, qr_input)

    # Apply the quantum Fourier transform to the input register
    qc.append(QFT(n, inverse=False), qr_input)

    # Measure the input register
    qc.measure(qr_input, cr)

    return qc

# Example function to test
def f(x):
    return x 

# Create the circuit to detect the period of f
qc = period_finding_circuit(f)

# Simulate the circuit
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
counts = job.result().get_counts()

print(counts)

qc = period_finding_circuit(f)
print(qc.draw())



{'0': 1024}
       ┌───┐┌─────────┐┌─────┐┌─┐
  q48: ┤ H ├┤ Unitary ├┤ QFT ├┤M├
       └───┘└─────────┘└─────┘└╥┘
  q49: ────────────────────────╫─
                               ║ 
c10: 1/════════════════════════╩═
                               0 
